# Coursera Capstone project
### this Notebook will be used to work on Coursera Capstone project.

In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
# Scrape url for table
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(url)
soup = BeautifulSoup(req.content,'html.parser')

table = soup.find('table', attrs={'class':'wikitable sortable'})

In [3]:
# Create DataFrame from scraped data
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=["PostalCode", "Borough", 'Neighborhood'])

In [4]:
df.drop(0, inplace=True, axis=0)

In [5]:
# Clean data
df['PostalCode'] = [x.replace('\n','') for x in df['PostalCode']]
df['Borough'] = [x.replace('\n','') for x in df['Borough']]
df['Neighborhood'] = [x.replace('\n','') for x in df['Neighborhood']]
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#Drop rows if Borough is Not assigned
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
# Reset DataFrame index
df.reset_index(drop=True, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
# Print DataFrame shape
print(df.shape)

(103, 3)


In [9]:
import geocoder # import geocoder

In [20]:
Latitude = []
Longitude = []

for n in df['PostalCode']: 
# initialize your variable to None
    lat_lng_coords = None

# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(n))
        lat_lng_coords = g.latlng

        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        Latitude.append(latitude)
        Longitude.append(longitude)

TypeError: 'NoneType' object is not subscriptable

In [13]:
# Import Coordinates CSV
coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
Latitude = []
Longitude = []
for n in df['PostalCode']:
    lat = coordinates[coordinates['Postal Code'] == n]['Latitude']
    lng = coordinates[coordinates['Postal Code'] == n]['Longitude']

    Latitude.append(lat.values[0])
    Longitude.append(lng.values[0])


In [31]:
df['Latitude'] = Latitude
df['Longitude'] = Longitude
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
